### Installation of required libs for Gemini and PaLM


In [7]:
# !pip install streamlit

In [9]:
!pip install --upgrade google-cloud-aiplatform

In [3]:
!pip install --upgrade google-cloud-aiplatform
GCP_PROJECT= PROJECT_ID=project_id='my-project-0004-346516'
LOCATION = REGION = 'us-central1'

### Vertex AI setup

In [4]:
import os

# import streamlit as st
import vertexai
from vertexai.preview.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)


# @st.cache_resource
def get_model():
    generation_model = TextGenerationModel.from_pretrained("text-bison@002")
    return generation_model


def get_text_generation(prompt="", **parameters):
    generation_model = get_model()
    response = generation_model.predict(prompt=prompt, **parameters)

    return response.text

In [5]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

# input_prompt = """can you give me details of paracetamol"""

def generate(input_prompt):
    model = GenerativeModel("gemini-ultra")
    responses = model.generate_content(
        input_prompt ,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 32
    },
        safety_settings=[],
        stream=True,
    )
    
    all_response  = []
    
    for response in responses:
        # print(response.text, end="")
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))
    

def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1
    },stream=True,)
    
    all_response  = []
    
    for response in responses:
        all_response.append(response.text)
    
    # print (all_response)
    
    return(" ".join(all_response))


In [6]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_k": 40
}

def generate_palm_unicorn_v1(input_prompt):
    
    model = TextGenerationModel.from_pretrained("text-unicorn@001")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)

def generate_palm_bison32k(input_prompt):
    
    model = TextGenerationModel.from_pretrained("text-bison-32k")

    response = model.predict(
        input_prompt,
        **parameters
    )
    print(f"Response from Model: {response.text}")
    
    return(response.text)


In [9]:
input_prompt = "What are the symptoms of influenza?" 

# generate_pro(input_prompt)
# generate_palm_unicorn_v1(input_prompt)
# generate_palm_bison32k(input_prompt)

In [10]:
!ls

'$BUCKET_URI'				  questions-Copy1.json
 RAG_on_DBCreated_part3.ipynb		  questions.json
 Testing_Gemini+Palm_models_part4.ipynb   questions_2.json
 Winnie_the_Pooh_3_Pages.pdf		 'reading pdf_part1.ipynb'
 diapers.png				 'reading pdf_part2.ipynb'
 harry_potte_qa.csv			  requirement2.txt
 product-embs.json			  requirements.txt


### Read the Q&A file

#### This uses the file from Matching Engine which has questions and retrieved document results

In [21]:
import pandas as pd
filename = "./harry_potte_qa_output.csv"
df_qa = pd.read_csv(filename, sep ="|")

# print(df_qa.head(1))
System_Prompts = """ You are an expert in reading harry potter books, but only provide evidences from the information provide and do not use an other information
so here are some search results : 
"""

Question_Prompts = """ -- Based on information above help to answer following user question
"""

df_qa['combine_prompt_RAG1'] = System_Prompts + ' ' +df_qa['pagewise_texts_v1'] + ' Question_Prompts : '+ df_qa['Question'] 
df_qa['combine_prompt_RAG2'] = System_Prompts + ' ' +df_qa['pagewise_texts_v2'] + ' Question_Prompts : '+ df_qa['Question'] 
df_qa['combine_prompt_RAG3'] = System_Prompts + ' ' +df_qa['pagewise_texts_v3'] + ' Question_Prompts : '+ df_qa['Question'] 


# print(df['System Prompts'], df['RAG Results'] ,df['User Question'] )
# print(selected_column[0])

In [22]:
# generate_medlpalm(df.loc[0 ,'combine_prompt'])

In [24]:
for i in range(0, len(df_qa)):

    if i<=100:
        # df['Gemini_ultra_model_output'][i] = generate(df['combine_prompt'][i])
        print("iteration #", i)

    df_qa.loc[i, "Gemini_pro_model_output_v1"] = generate_pro(df_qa.loc[i,'combine_prompt_RAG1'])
    df_qa.loc[i, "Gemini_pro_model_output_v2"] = generate_pro(df_qa.loc[i,'combine_prompt_RAG2'])
    df_qa.loc[i, "Gemini_pro_model_output_v3"] = generate_pro(df_qa.loc[i,'combine_prompt_RAG3'])

    # df.loc[i, "medLM_model_output"] = generate_medllms_v1(df.loc[i,'combine_prompt'])
    df_qa.loc[i, "palm_bison32k_output_v1"] = generate_palm_bison32k(df_qa.loc[i,'combine_prompt_RAG1'])
    df_qa.loc[i, "palm_bison32k_output_v2"] = generate_palm_bison32k(df_qa.loc[i,'combine_prompt_RAG2'])
    df_qa.loc[i, "palm_bison32k_output_v3"] = generate_palm_bison32k(df_qa.loc[i,'combine_prompt_RAG3'])

# generate_medllms_v1(input_prompt)
# generate_palm_unicorn_v1(input_prompt)
# input_prompt = "What are the symptoms of influenza?" 
# generate_medlpalm(input_prompt)    
    
# print( "/n output here ::" , df['Gemini_ultra_model_output'][i])
# df = df.assign(Gemini_ultra_model_output=generate(df.combine_prompt))
# df['combine_prompt'].head(3)

# df['Gemini_ultra_model_output'].head(3)


iteration # 0
Response from Model:  This context does not mention anything about Harry Potter or magical creatures, so I cannot answer this question from the provided information.
Response from Model:  The provided text does not contain any information about Harry Potter or any magical creatures.
Response from Model:  The provided information does not contain the answer to the question.
iteration # 1
Response from Model:  The provided text does not contain any information about the name of the school newspaper at Hogwarts.
Response from Model:  This information is not available from the context provided.
Response from Model:  The provided text does not contain any information about the school newspaper at Hogwarts.
iteration # 2
Response from Model:  The provided text does not contain any information about the magical map of Hogwarts.
Response from Model:  The provided text does not mention anything about a magical map of Hogwarts, so I cannot answer this question.
Response from Model:

In [25]:
df_qa.columns

Index(['Question', 'Answer', 'i', 'j', 'k', 'pagewise_texts_v1',
       'pagewise_texts_v2', 'pagewise_texts_v3', 'splitted_texts_v1',
       'splitted_texts_v2', 'splitted_texts_v3', 'splitted_texts_chunks_v1',
       'splitted_texts_chunks_v2', 'splitted_texts_chunks_v3', 'page_id_v1',
       'page_id_v2', 'page_id_v3', 'combine_prompt_RAG1',
       'combine_prompt_RAG2', 'combine_prompt_RAG3',
       'Gemini_ultra_model_output_v1', 'Gemini_ultra_model_output_v2',
       'Gemini_ultra_model_output_v3', 'palm_bison32k_output_v1',
       'palm_bison32k_output_v2', 'palm_bison32k_output_v3'],
      dtype='object')

In [26]:

# Delete the 'col2' column
df_qa = df_qa.drop('combine_prompt_RAG1', axis=1)
df_qa = df_qa.drop('combine_prompt_RAG2', axis=1)
df_qa = df_qa.drop('combine_prompt_RAG3', axis=1)

output1 = "./harry_potte_qa_model_out.csv"

df_qa.to_csv(output1)


In [27]:
df_qa.head(1)

,Question,Answer,i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,...,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3,Gemini_ultra_model_output_v1,Gemini_ultra_model_output_v2,Gemini_ultra_model_output_v3,palm_bison32k_output_v1,palm_bison32k_output_v2,palm_bison32k_output_v3
0,What is the name of the magical creature that ...,Thestral,9,8,19,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,...,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",2,1,3,The provided text does not contain any informa...,The provided text does not contain any informa...,The provided context does not contain any info...,This context does not mention anything about ...,The provided text does not contain any inform...,The provided information does not contain the...


### Lets now call the Eval function to understand which models were good and which werent so good